In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchinfo
import onnx
import torch.onnx
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import struct
import numpy as np

In [94]:
class SimpleNet(nn.Module):
    def __init__(self, num_classes=10, init_weights=True):
        super(SimpleNet, self).__init__()
        # self.conv1 = nn.Conv2d(in_channels=2, out_channels=4, kernel_size=3, stride=2, bias=True)
        # self.conv2 = nn.Conv2d(in_channels=3, out_channels=5, kernel_size=4, stride=2)
        # self.conv3 = nn.Conv2d(in_channels=5, out_channels=5, kernel_size=4, stride=2)

        self.linear1 = nn.Linear(in_features=3, out_features=4, bias=False)
        # self.linear2 = nn.Linear(in_features=10, out_features=num_classes)

    def forward(self, x):
        # x = self.conv1(x)
        x = self.linear1(x)
        return x

model = SimpleNet()
model = model.cuda() if torch.cuda.is_available() else model.cpu()

# x = torch.randn(1, 2, 5, 5, requires_grad=True)
x = torch.randn(5, 3, requires_grad=True)
x = x.cuda() if torch.cuda.is_available() else x.cpu()

# Export the model
torch.onnx.export(model,                     # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "model.onnx",              # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=13,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
)

def onnx_check_model(onnx_model):
    try:
        onnx.checker.check_model(onnx_model)
    except onnx.checker.ValidationError as e:
        print('The model is invalid: %s' % e)
    else:
        print('The model is valid!')

model_onnx = onnx.load("model.onnx")
onnx_check_model(model_onnx)

The model is valid!


In [95]:
model_onnx.graph.initializer

[dims: 3
dims: 4
data_type: 1
name: "onnx::MatMul_4"
raw_data: "X\270\206\275\301\312y\276\345\036\254>\320;\235\275.\307v\275<.$=\033\3673\276~=\007>iX\351<\367I\327=\356\3701\276\0203\347>"
]

In [96]:
model_onnx.graph.node

[input: "input"
input: "onnx::MatMul_4"
output: "output"
name: "MatMul_0"
op_type: "MatMul"
]

In [101]:
from converter.converter import parse_onnx_model, dump_onnx_model
from converter.helpers import TYPEDEF

%load_ext autoreload
%autoreload 2

ir = parse_onnx_model(model_onnx)
dump_onnx_model(ir, "ir.bin")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [99]:
ir

OrderedDict([(0,
              {'name': 'input',
               'op_type': INPUT,
               'inputs': [],
               'outputs': [1],
               'dims': [5, 3]}),
             (1,
              {'name': 'MatMul_0',
               'op_type': LINEAR,
               'initializer': {'weight': {'name': 'onnx::MatMul_4',
                 'raw_data': b'X\xb8\x86\xbd\xc1\xcay\xbe\xe5\x1e\xac>\xd0;\x9d\xbd.\xc7v\xbd<.$=\x1b\xf73\xbe~=\x07>iX\xe9<\xf7I\xd7=\xee\xf81\xbe\x103\xe7>',
                 'dims': [3, 4],
                 'data_type': FLOAT},
                'bias': {}},
               'data_layout': None,
               'inputs': [0],
               'outputs': [2]})])